In [73]:
import exploring_script as es
import importlib
importlib.reload(es)

import numpy as np
import pandas as pd
import astropy.units as u
import time

from IPython.display import display, Markdown

from fink_fat.seeding.dbscan_seeding import intra_night_seeding, seeding_purity, seeding_completude
from fink_fat.associations.intra_night_association import intra_night_association, new_trajectory_id_assignation

In [74]:
confirmed_sso = es.load_data(columns=["ssnamenr", "fid", "ra", "dec", "magpsf", "sigmapsf", "jd", "nid", "candid"])

In [75]:
night_id = 1718
night_obs = confirmed_sso[confirmed_sso["nid"] == night_id]

In [76]:
f"nb alerts in this night: {len(night_obs)}"

'nb alerts in this night: 39496'

In [6]:
intra_night_assoc = intra_night_seeding(night_obs)

In [7]:
seeding_purity(intra_night_assoc)

90.44811320754717

In [8]:
seeding_completude(intra_night_assoc)

99.96630727762803

## Find the best separation limit

In [25]:
sep_lim_test = np.linspace(1.8, 2.8, 25) * u.arcmin

purity_dict = {}
completude_dict = {}
i = 1
nb_it = 50

In [26]:
sep_lim_test

<Quantity [1.8       , 1.84166667, 1.88333333, 1.925     , 1.96666667,
           2.00833333, 2.05      , 2.09166667, 2.13333333, 2.175     ,
           2.21666667, 2.25833333, 2.3       , 2.34166667, 2.38333333,
           2.425     , 2.46666667, 2.50833333, 2.55      , 2.59166667,
           2.63333333, 2.675     , 2.71666667, 2.75833333, 2.8       ] arcmin>

In [27]:
for nid in np.random.choice(confirmed_sso["nid"].unique(), nb_it):
    print(f"--- {nid} ---")
    tmp_pdf = confirmed_sso[confirmed_sso["nid"] == nid]
    print(f"nb alerts: {len(tmp_pdf)}")
    list_pure = []
    list_compl = []
    t_0 = time.time()
    for sep in sep_lim_test.to(u.deg):
        cluster_night = intra_night_seeding(tmp_pdf, sep.value)
        pure = seeding_purity(cluster_night)
        completude = seeding_completude(cluster_night)

        list_pure.append(pure)
        list_compl.append(completude)

    purity_dict[nid] = list_pure
    completude_dict[nid] = list_compl

    print(f"elapsed time: {time.time() - t_0}")
    print(f"remaining: {nb_it - i}")
    i += 1
    print()

pure_pdf = pd.DataFrame.from_dict(purity_dict).T
pure_pdf.columns = ["sep_lim_{:.2f}".format(el.value) for el in sep_lim_test]
pure_pdf.to_parquet("dbscan_purity.parquet")

completude_pdf = pd.DataFrame.from_dict(completude_dict).T
completude_pdf.columns = ["sep_lim_{:.2f}".format(el.value) for el in sep_lim_test]
completude_pdf.to_parquet("dbscan_completude.parquet")

--- 1039 ---
nb alerts: 25328
elapsed time: 13.313493967056274
remaining: 49

--- 2032 ---
nb alerts: 20828
elapsed time: 12.979475259780884
remaining: 48

--- 1721 ---
nb alerts: 1114
elapsed time: 0.7994608879089355
remaining: 47

--- 1563 ---
nb alerts: 29280
elapsed time: 17.652008533477783
remaining: 46

--- 1470 ---
nb alerts: 30835
elapsed time: 20.826679229736328
remaining: 45

--- 1932 ---
nb alerts: 244
elapsed time: 0.560661792755127
remaining: 44

--- 2016 ---
nb alerts: 13112
elapsed time: 9.86624002456665
remaining: 43

--- 1413 ---
nb alerts: 44213
elapsed time: 28.51993227005005
remaining: 42

--- 2105 ---
nb alerts: 17553
elapsed time: 12.140409469604492
remaining: 41

--- 1703 ---
nb alerts: 291
elapsed time: 0.5704402923583984
remaining: 40

--- 1306 ---
nb alerts: 23708
elapsed time: 17.778419017791748
remaining: 39

--- 2152 ---
nb alerts: 46116
elapsed time: 34.542004346847534
remaining: 38

--- 1886 ---
nb alerts: 43779
elapsed time: 32.429908990859985
remaining:

In [28]:
pure_pdf = pd.read_parquet("dbscan_purity.parquet")
completude_pdf = pd.read_parquet("dbscan_completude.parquet")

In [29]:
display(Markdown(pure_pdf.describe().to_markdown()))

|       |   sep_lim_1.80 |   sep_lim_1.84 |   sep_lim_1.88 |   sep_lim_1.93 |   sep_lim_1.97 |   sep_lim_2.01 |   sep_lim_2.05 |   sep_lim_2.09 |   sep_lim_2.13 |   sep_lim_2.17 |   sep_lim_2.22 |   sep_lim_2.26 |   sep_lim_2.30 |   sep_lim_2.34 |   sep_lim_2.38 |   sep_lim_2.42 |   sep_lim_2.47 |   sep_lim_2.51 |   sep_lim_2.55 |   sep_lim_2.59 |   sep_lim_2.63 |   sep_lim_2.67 |   sep_lim_2.72 |   sep_lim_2.76 |   sep_lim_2.80 |
|:------|---------------:|---------------:|---------------:|---------------:|---------------:|---------------:|---------------:|---------------:|---------------:|---------------:|---------------:|---------------:|---------------:|---------------:|---------------:|---------------:|---------------:|---------------:|---------------:|---------------:|---------------:|---------------:|---------------:|---------------:|---------------:|
| count |       48       |       48       |       48       |       48       |        48      |        48      |       48       |       48       |       48       |       48       |       48       |       48       |       48       |       48       |       48       |       48       |        48      |       48       |       48       |       48       |       48       |       48       |       48       |       48       |       48       |
| mean  |       96.0034  |       96.008   |       96.0366  |       96.0126  |        96.0039 |        95.9941 |       95.9695  |       95.9309  |       95.8863  |       95.824   |       95.7593  |       95.6844  |       95.5956  |       95.4845  |       95.3786  |       95.2879  |        95.1782 |       95.0538  |       94.9419  |       94.8222  |       94.6926  |       94.5171  |       94.3916  |       94.2266  |       94.0786  |
| std   |        2.94811 |        2.79512 |        2.65461 |        2.54146 |         2.4595 |         2.3965 |        2.34704 |        2.31661 |        2.30996 |        2.31757 |        2.32611 |        2.34584 |        2.38636 |        2.44063 |        2.50166 |        2.55971 |         2.6314 |        2.70099 |        2.74765 |        2.80823 |        2.87527 |        2.86324 |        2.92087 |        2.91549 |        2.98731 |
| min   |       86.079   |       86.7009  |       87.2682  |       87.89    |        88.5446 |        89.0356 |       89.5702  |       89.9084  |       90.3229  |       90.6393  |       90.9884  |       91.1303  |       91.163   |       91.1084  |       91.0866  |       91.0539  |        90.9557 |       90.8106  |       90.6228  |       90.3599  |       90.0595  |       89.8341  |       89.5962  |       89.3959  |       89.1142  |
| 25%   |       95.1184  |       95.062   |       94.8358  |       94.7218  |        94.5741 |        94.4654 |       94.4752  |       94.3835  |       94.1485  |       94.0133  |       93.8416  |       93.7399  |       93.6489  |       93.3701  |       93.1765  |       93.0368  |        92.8803 |       92.7369  |       92.5884  |       92.4182  |       92.34    |       92.1545  |       91.9702  |       91.8195  |       91.6122  |
| 50%   |       96.3994  |       96.286   |       96.2068  |       96.0723  |        95.932  |        95.9467 |       95.9434  |       95.8376  |       95.6782  |       95.5138  |       95.4692  |       95.441   |       95.4812  |       95.2557  |       95.1147  |       95.0445  |        94.9059 |       94.7391  |       94.7076  |       94.6057  |       94.4938  |       94.4134  |       94.2147  |       94.069   |       93.8872  |
| 75%   |       97.7093  |       97.6392  |       97.6276  |       97.6172  |        97.7086 |        97.6945 |       97.7265  |       97.6775  |       97.6007  |       97.4446  |       97.3156  |       97.2608  |       97.1846  |       97.0806  |       96.8738  |       96.7499  |        96.7493 |       96.6884  |       96.5709  |       96.5667  |       96.3994  |       96.2853  |       96.3054  |       96.2605  |       96.0696  |
| max   |      100       |      100       |      100       |      100       |       100      |       100      |      100       |      100       |      100       |      100       |      100       |      100       |      100       |      100       |      100       |      100       |       100      |      100       |      100       |      100       |      100       |      100       |      100       |      100       |      100       |

In [31]:
display(Markdown(completude_pdf.describe().to_markdown()))

|       |   sep_lim_1.80 |   sep_lim_1.84 |   sep_lim_1.88 |   sep_lim_1.93 |   sep_lim_1.97 |   sep_lim_2.01 |   sep_lim_2.05 |   sep_lim_2.09 |   sep_lim_2.13 |   sep_lim_2.17 |   sep_lim_2.22 |   sep_lim_2.26 |   sep_lim_2.30 |   sep_lim_2.34 |   sep_lim_2.38 |   sep_lim_2.42 |   sep_lim_2.47 |   sep_lim_2.51 |   sep_lim_2.55 |   sep_lim_2.59 |   sep_lim_2.63 |   sep_lim_2.67 |   sep_lim_2.72 |   sep_lim_2.76 |   sep_lim_2.80 |
|:------|---------------:|---------------:|---------------:|---------------:|---------------:|---------------:|---------------:|---------------:|---------------:|---------------:|---------------:|---------------:|---------------:|---------------:|---------------:|---------------:|---------------:|---------------:|---------------:|---------------:|---------------:|---------------:|---------------:|---------------:|---------------:|
| count |       48       |        48      |       48       |       48       |       48       |       48       |       48       |        48      |      48        |      48        |      48        |      48        |      48        |      48        |      48        |      48        |      48        |      48        |      48        |      48        |      48        |      48        |      48        |      48        |      48        |
| mean  |       98.5374  |        98.6806 |       98.8184  |       98.9396  |       99.0436  |       99.1378  |       99.2267  |        99.297  |      99.3732   |      99.4422   |      99.4993   |      99.5501   |      99.5928   |      99.6256   |      99.6553   |      99.6831   |      99.7011   |      99.7197   |      99.7419   |      99.7581   |      99.7759   |      99.7917   |      99.8096   |      99.8191   |      99.8259   |
| std   |        2.28283 |         2.0626 |        1.84259 |        1.65143 |        1.47592 |        1.31493 |        1.16613 |         1.03   |       0.899143 |       0.777475 |       0.672139 |       0.587262 |       0.522391 |       0.464098 |       0.424621 |       0.380542 |       0.358691 |       0.331609 |       0.306684 |       0.285816 |       0.266863 |       0.249282 |       0.225406 |       0.211973 |       0.204747 |
| min   |       89.6465  |        90.4757 |       91.2285  |       92.134   |       92.974   |       93.6941  |       94.4032  |        95.0578 |      95.6906   |      96.3343   |      96.9016   |      97.3162   |      97.6107   |      97.9271   |      98.2108   |      98.4107   |      98.4107   |      98.4743   |      98.5378   |      98.665    |      98.7285   |      98.8557   |      99.11     |      99.1736   |      99.1736   |
| 25%   |       98.3592  |        98.5348 |       98.6481  |       98.832   |       98.9511  |       98.9624  |       99.0823  |        99.1616 |      99.3457   |      99.3847   |      99.4198   |      99.4297   |      99.4297   |      99.4473   |      99.4934   |      99.4966   |      99.5062   |      99.5717   |      99.6494   |      99.662    |      99.6684   |      99.6748   |      99.7146   |      99.7184   |      99.7431   |
| 50%   |       99.4731  |        99.4957 |       99.5373  |       99.5681  |       99.5984  |       99.7107  |       99.7376  |        99.7376 |      99.769    |      99.7818   |      99.7886   |      99.8025   |      99.812    |      99.8302   |      99.8386   |      99.8386   |      99.8395   |      99.8497   |      99.8528   |      99.8612   |      99.8684   |      99.8797   |      99.8951   |      99.9028   |      99.9062   |
| 75%   |       99.8113  |        99.8229 |       99.841   |       99.841   |       99.841   |       99.8513  |       99.8622  |        99.8622 |      99.8778   |      99.8826   |      99.9056   |      99.9338   |      99.949    |      99.949    |      99.949    |      99.949    |      99.949    |      99.949    |      99.949    |      99.9647   |      99.9652   |      99.9677   |      99.9771   |      99.9771   |      99.9771   |
| max   |      100       |       100      |      100       |      100       |      100       |      100       |      100       |       100      |     100        |     100        |     100        |     100        |     100        |     100        |     100        |     100        |     100        |     100        |     100        |     100        |     100        |     100        |     100        |     100        |     100        |

In [40]:
print("min-max completude: {:.4f}".format(sep_lim_test[completude_pdf.describe().T['min'].argmax()]))
print("min-max purity: {:.4f}".format(sep_lim_test[pure_pdf.describe().T['min'].argmax()]))

min-max completude: 2.7583 arcmin
min-max purity: 2.3000 arcmin


We choose the min-max purity, we prefer to keep pure cluster while catching all the asteroids. The system can always catch the asteroids later.

In [41]:
best_sep_dbscan = 2.3 * u.arcmin

## Comparison with the old intra night association

In [98]:
old_res = {}
dbscan_res = {}

nb_alert_list = []
purity_old_list = []
comple_old_list = []
time_old_list = []

purity_dbscan_list = []
comple_dbscan_list = []
time_dbscan_list = []

nb_it = 50
random_night = np.random.choice(confirmed_sso["nid"].unique(), nb_it)


for night_id in random_night:
    print(f"--- {night_id} ---")
    night_obs = confirmed_sso[confirmed_sso["nid"] == night_id]
    print(f"nb alerts: {len(night_obs)}")
    nb_alert_list.append(len(night_obs))


    print("# start old intra night assoc")
    t_start_old = time.time()
    left, right = intra_night_association(night_obs, best_sep_dbscan.to(u.arcsecond))
    intra_night_old = new_trajectory_id_assignation(left, right, 0).rename({"trajectory_id": "cluster"}, axis=1)
    time_old = time.time() - t_start_old

    if len(intra_night_old) != 0:
        purity_old = seeding_purity(intra_night_old)
        completude_old = seeding_completude(intra_night_old)
        purity_old_list.append(purity_old)
        comple_old_list.append(completude_old)
        time_old_list.append(time_old)

    print("# start dbscan")
    t_start_dbscan = time.time()
    intra_night_dbscan = intra_night_seeding(night_obs, best_sep_dbscan.to(u.deg).value)
    time_dbscan = time.time() - t_start_dbscan

    purity_dbscan = seeding_purity(intra_night_dbscan)
    completude_dbscan = seeding_completude(intra_night_dbscan)
    purity_dbscan_list.append(purity_dbscan)
    comple_dbscan_list.append(completude_dbscan)
    time_dbscan_list.append(time_dbscan)
    print()
    print()


old_res_pdf = pd.DataFrame.from_dict({
    "time": time_old_list,
    "purity": purity_old_list,
    "completude": comple_old_list,
    "nid": random_night
})


dbscan_res_pdf = pd.DataFrame.from_dict({
    "time": time_dbscan_list,
    "purity": purity_dbscan_list,
    "completude": comple_dbscan_list,
    "nid": random_night
})

--- 1311 ---
nb alerts: 7120
# start old intra night assoc
# start dbscan


--- 2032 ---
nb alerts: 20828
# start old intra night assoc
# start dbscan


--- 2091 ---
nb alerts: 19223
# start old intra night assoc
# start dbscan


--- 2144 ---
nb alerts: 33321
# start old intra night assoc
# start dbscan


--- 1506 ---
nb alerts: 18875
# start old intra night assoc
# start dbscan


--- 2181 ---
nb alerts: 23627
# start old intra night assoc
# start dbscan


--- 1457 ---
nb alerts: 4251
# start old intra night assoc
# start dbscan


--- 1685 ---
nb alerts: 23823
# start old intra night assoc
# start dbscan


--- 1547 ---
nb alerts: 8516
# start old intra night assoc
# start dbscan


--- 1538 ---
nb alerts: 32866
# start old intra night assoc
# start dbscan


--- 2055 ---
nb alerts: 25668
# start old intra night assoc
# start dbscan


--- 1418 ---
nb alerts: 19182
# start old intra night assoc
# start dbscan


--- 1112 ---
nb alerts: 9645
# start old intra night assoc
# start dbscan


---

In [99]:
old_res_pdf.describe()

,time,purity,completude,nid
count,50.000000,50.000000,50.000000,50.000000
mean,12.857046,99.462762,99.577608,1668.920000
std,13.100567,0.389937,0.305481,341.911126
min,0.116840,98.057903,98.624348,1048.000000
25%,4.245540,99.293882,99.413541,1419.250000
50%,8.401978,99.474435,99.606675,1636.500000
75%,17.074954,99.739284,99.813371,2005.000000
max,59.498991,100.000000,100.000000,2181.000000


In [100]:
dbscan_res_pdf.describe()

,time,purity,completude,nid
count,50.000000,50.000000,50.000000,50.000000
mean,0.050329,95.607959,99.489212,1668.920000
std,0.035710,2.038612,1.139934,341.911126
min,0.004375,88.549618,92.046294,1048.000000
25%,0.021979,94.467442,99.495421,1419.250000
50%,0.045358,95.648534,99.767933,1636.500000
75%,0.072283,97.117460,99.917099,2005.000000
max,0.147985,100.000000,100.000000,2181.000000
